### import

In [1]:
import pandas as pd
import numpy as np
import pickle
import re
import os
from os.path import join
from tqdm import tqdm
from datetime import datetime
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl import Workbook
import warnings
warnings.simplefilter("ignore")
import functions
from typing import Optional

# 출력 옵션
pd.options.display.float_format = '{:,.0f}'.format

# dtype 정의
debt_dtype = {'채무자키':str, '타채무자키':str, '담당자키':str, '관리자비고':str}
account_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str}
grt_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str, '보증인키':str}
event_dtype = {'채무자키':str, '법조치키':str, '계좌키':str, '관련법조치키':str, '법취하키':str, '타법조치키':str, '타채무자키':str, '관할법원코드':str}

def fill_groupwise(df:pd.DataFrame, column:str, group_cols: Optional[list[str]] = None) -> pd.DataFrame :
    """새채무자키와 사건번호(default)를 group_cols로하여 column에서 빈값을 동일 그룹의 앞뒤값으로 채워주는 함수
    column의 빈문자열을 na로 바꿔주는 코드 내포함"""
    if group_cols is None : group_cols = ["새채무자키", "사건번호"]
    df[column] = df[column].replace("", pd.NA)
    df[column] = (
        df.groupby(group_cols)[column]
        .transform(lambda x: x.ffill().bfill())
    )
    return df

In [2]:
##################################
company = "솔림"      # 솔림 or 대성
basedate = "250331" 
##################################
기준일 = '20' + basedate[:2] +'-'+basedate[2:4]+'-'+basedate[4:6]

wd = join(r"D:\3.자산\전산 dataset", company)
comp_closing = {
    # 채권 자체의 소멸
    "종료" : r"(?<!청산)종결(?!요청)|매각(?!예정)|(?<!피)환매(?!예정)|DS|완납|완제|종료",
    # 비정상-종결요청
    "종결요청" : r"종결요청|매각\(예정\)|(?<!피)환매\(예정\)|패소",
    # 시효
    "시효" : r"시효|완성|만료", # 부활 제외함 231222
    # 채무자별로 체크
    "상각" : r"상각", # 면책포함여부보다 먼저 검색되어야 함. 실사한 것인고, 담당자에만 있으므로 차주 파산(확정) 이런 거보다 우선함. 대성에서 원래 상각이었던 거는 내담당으로 바꿈. 솔림대성 모두 실사한 것만 담당자 상각임
    "면책" : r"면책|면탈",  
    "불가" : r"추심제외|포기|파산(?!(\(기각\)|\s누락|\(누락\)))|사망(?!\(상속\))|폐업|청산종결|패소", # 관리채권 중 추심제외,포기 없음 확인. 추심제외는 다시 살림(해당채무자만 제외한다는 의미)250103 # 담당자 누락제외는 채무상태에 적용하면 안 됨.(계좌의 메모를 봐야함)
    "공채권" : r"공채권"
    # 이민,국적상실, (말소), 실종, 거주불명, (채권존부)소송중은 정상으로
    }

### 파일읽기

In [3]:
# 파일 읽기
###################
ext = ".pkl"  
###################
# 파일검색어
파일검색어 = ["채무자조회새창_", "계좌조회새창_", "보증인새창_", "법조치조회새창"] # 메모는 따로 하자

# 기본 경로
base_dir = join(wd, basedate)
if ext == ".pkl" : 
    base_dir = join(base_dir, "pkl")
    
# 파일명리스트
files = []
for i in range(len(파일검색어)) : 
    if re.search("보증인새창", 파일검색어[i]) : # 새보증인상태 파일 말고 원 파일 읽자
        files.append(os.path.splitext(functions.키워드로파일명찾기(base_dir, 파일검색어[i],'새보증인상태',전체경로=False))[0])        
    else : 
        files.append(os.path.splitext(functions.키워드로파일명찾기(base_dir, 파일검색어[i],전체경로=False))[0])

# 채무자, 계좌, 보증인, 개회신복 기준데이터->차주와 보증인으로 나누 저장
if ext == ".pkl" : # fillna()는 엑셀읽을 때 미리 해둬야함, 조정, 개회 진행중 시트만 읽어서 저장한 경우
    debt_ori = pd.read_pickle(join(wd, basedate, "pkl", files[0]+ext))
    account_ori = pd.read_pickle(join(wd, basedate, "pkl", files[1]+ext))
    grt_ori = pd.read_pickle(join(wd, basedate, "pkl", files[2]+ext))
    event_ori = pd.read_pickle(join(wd, basedate, "pkl", files[3]+ext))
else : 
    debt_ori = pd.read_excel(join(wd, basedate, files[0]+ext), dtype=debt_dtype).fillna("")
    
    # 새채무자키 없으면 일부러 오류나게
    if len(debt_ori.query("관리자비고==''")) > 1 :
        print("새채무자키 부여 필요")
        print(len(nothing)) # 아래 코드 실행안 되도록 오류를 위한 코드

    account_ori = pd.read_excel(join(wd, basedate, files[1]+ext), dtype=account_dtype).fillna("")
    grt_ori = pd.read_excel(join(wd, basedate, files[2]+ext), dtype=grt_dtype).fillna("")
    event_ori = pd.read_excel(join(wd, basedate, files[5]+ext), dtype=event_dtype).fillna("")

### 재시작점 및 기본변수

In [4]:
debt = debt_ori.copy()
account = account_ori.copy()
grt = grt_ori.copy()
event = event_ori.copy()

# 새채무자키
debt = debt.rename(columns={'관리자비고':'새채무자키'})
새채무자키목록 = debt[["채무자키","새채무자키"]]
account = account.merge(새채무자키목록, on='채무자키', how='left')
grt = grt.merge(새채무자키목록, on='채무자키', how='left')
event = event.merge(새채무자키목록, on='채무자키', how='left')


# 이름키, 이름키구분 추가 : 법조치는 채무자키 아니라 새채무자키를 써야함
# ***법조치대상자명과 전산채무자명에서 같은것을 다르게 표현할 수 있는 것은 삭제한다.(통일할까 했으나, 있고 없을수도 있으니 삭제)
# ***(주 ), ( 주)는 없음을 확인함
    # 현재 채무자
debt["이름키_"] = debt["새채무자키"] + debt["성명"].str.replace(r"(㈜|\(주\))","", regex=True)
debt["이름키구분"] = "채무자"
    # 변경전 채무자
변경전debt = debt.query('변경전성명인!=""')
변경전debt["이름키_"] = 변경전debt["새채무자키"] + 변경전debt["변경전성명인"].str.replace(r"(㈜|\(주\))","", regex=True)
변경전debt["이름키구분"] = "변경전채무자"
    # 보증인
grt["이름키_"] = grt["새채무자키"] + grt["보증인성명인"].str.replace(r"(㈜|\(주\))","", regex=True)
grt["이름키구분"] = "보증인"

account["이름키_"] = account["새채무자키"] + account["채무자명"].str.replace(r"(㈜|\(주\))","", regex=True)

# 대상자, 대상자구분 (다계좌에 따른 중복값 제거)
def remove_duplicates_in_cell(cell):
    return ','.join(sorted(set(cell.split(','))))
event['대상자구분'] = event['대상자구분'].apply(remove_duplicates_in_cell)
event['대상자명'] = event['대상자명'].apply(remove_duplicates_in_cell)

event["이름키_"] = event["새채무자키"] + event["대상자명"].str.replace(r"(㈜|\(주\))","", regex=True)

# 전체 이름키
전체이름키 = pd.concat([
    debt[["이름키_","이름키구분"]], 변경전debt[["이름키_","이름키구분"]], grt[["이름키_","이름키구분"]]
    ])
    
    # 중복제거
전체이름키 = 전체이름키.drop_duplicates()


# 미종결새채무자키(정상계좌를 하나라도 포함하는 새채무자키 sr)
계좌종결담당자정규식 = r"(?<!청산)종결|매각(?!예정)|(?<!피)환매(?!예정)|DS|완납|완제|종료|매각\(예정\)|(?<!피)환매\(예정\)|상각" 
미종결새채무자키 = account.query('종결일=="" and ~담당자.str.contains(@계좌종결담당자정규식)')["새채무자키"]
미종결새채무자키 = 미종결새채무자키.drop_duplicates()

# 미종결보증인주민번호 : 보증인 면책은 확실한 거 (_붙은것만 본다), 파산면책은 제외
보증면책정규식 = re.compile(r"((?<!파산\()면책|패소|추심제외|포기|폐업|청산|종결|한정및포기|한정승인|상속포기).*_" )
미종결보증인주민번호 = grt[(grt.종결일=="") & ~grt.보증인상태.str.contains(보증면책정규식, regex=True, na=False)]["주민번호인"]
미종결보증인주민번호 = 미종결보증인주민번호.drop_duplicates()

# 사건번호 정규식
파산사건정규식 = re.compile(r'(\d{4,4}[하단면합 ]+\d+)')

### 법조치 파산사건 전처리

#### 1. 기본점검 : 종결, 법원사건번호, 법조치구분, 당사포함여부, 종국결과
- 대상자 구분은 따로 dc하기전에는 잘못된 게 많아서 엑셀출력

In [5]:
# 법조치에 등록된 파산사건번호
    # 새채무자키 추가
event = event.merge(debt[["채무자키","새채무자키"]])
    # 미종결
미종결event =event[event['새채무자키'].isin(미종결새채무자키)]
    # 파산사건
법조치cols = ["채무자키","새채무자키","이름키_","법조치키","대상자구분","대상자명","법조치구분","관할법원","사건번호","비고","법조치메모","종국결과","접수일","종료일","종국결과일","확정일","최신화일"]
법조치파산사건번호 = 미종결event.query('사건번호.str.contains("하")')[법조치cols]
    # 이름키구분열(이름이 현재 전산에 있는지 여부) [채무자, 보증인, 빈칸] -> 빈칸이면 해당 채무자키에 해당 이름을 가진 채무자가 없음.(개명/오타가능성 있으나 이건 무시하자..)
법조치파산사건번호 = 법조치파산사건번호.merge(전체이름키, on='이름키_', how='left') # 전체이름키 : 이름키, 이름키구분 열로 구성됨


# 법원사건번호 수정(df는 바로 수정함) - 거의 없으므로 보면서 바로 전산수정
    # 법원
print('(df수정완료)회생법원으로 수정할 목록')
display(법조치파산사건번호[법조치파산사건번호.관할법원.str.contains('수원지방|부산지방|서울(?!.*회생)')])
        # df 수정
법조치파산사건번호["관할법원"] = 법조치파산사건번호["관할법원"].replace({'수원지방법원':'수원회생법원','부산지방법원':'부산회생법원',r'서울.*지방법원':'서울회생법원'}, regex=True)

    # 사건번호
법조치파산사건번호['수정사건번호'] = 법조치파산사건번호["사건번호"].str.extract(파산사건정규식)
print('(df수정필요)사건번호가 정규식에 부합하지 않음')
display(법조치파산사건번호[법조치파산사건번호.수정사건번호 != 법조치파산사건번호.사건번호])
        # df 수정 (하단 -> 하면, 공백제거)
            # 1. 공백 제거 (모든 행 공통)
법조치파산사건번호["수정사건번호"] = 법조치파산사건번호["수정사건번호"].replace({r"\s+": ""}, regex=True)
            # 2. '하단'↔'하면'으로 변경 (상속재산파산 여부에 따라)
mask = 법조치파산사건번호["법조치구분"] == "상속재산파산"
법조치파산사건번호.loc[mask, "수정사건번호"] = (법조치파산사건번호.loc[mask, "수정사건번호"].replace({"하면": "하단"}, regex=True))
법조치파산사건번호.loc[~mask, "수정사건번호"] = (법조치파산사건번호.loc[~mask, "수정사건번호"].replace({"하단": "하면"}, regex=True))
            # 칼럼명, 위치 변경
법조치파산사건번호 = 법조치파산사건번호.rename(columns={'수정사건번호':'사건번호','사건번호':'수정전사건번호', '비고':'당사포함여부'})
법조치파산사건번호 = functions.swap_columns(법조치파산사건번호,'수정전사건번호','사건번호')
            # 보기 편하게 수정전후가 같은 겉은 수정전을 빈값으로
법조치파산사건번호["수정전사건번호"] = 법조치파산사건번호["수정전사건번호"].where(법조치파산사건번호["수정전사건번호"]!=법조치파산사건번호["사건번호"])


        # 상속재산파산인데 '하면'인건(상속재산파산은 하단으로 끝남)
print('(df수정필요,실사!)상속재산파산 사건 하면 -> 하단 변경할 것')
display(법조치파산사건번호[mask & (법조치파산사건번호.수정전사건번호.str.contains('하면'))])

# 당사포함여부 재작성[포함|누락|""]
conds = [법조치파산사건번호["당사포함여부"].str.contains('포함', na=False), 법조치파산사건번호["당사포함여부"].str.contains('누락', na=False)]
values = ["포함", "누락"]
법조치파산사건번호["당사포함여부"] = np.select(conds, values, default="")

# 종국결과 : 
# 하단 : (신청)취하, 기각 / 파산선고, 파산선고및동시폐지, 종결, 폐지(하면으로 넘어감)
# 하면 : (신청)취하, 기각 / 불허가 / 인용 ** 채무자의 : 채무자의 사망으로 종결
법조치파산사건번호['종국결과'].replace("", pd.NA, inplace=True)
    # 채무자의 -> 채무자의 사망으로 종료
법조치파산사건번호.loc[법조치파산사건번호.종국결과.str.contains('채무자의', na=False), '종국결과'] = "채무자의 사망으로 종료"
    # 종국결과가 정해진 값이 아닌 것
print('(df수정필요)목록에 없는 종국결과. 빈값은 아래서 엑셀출력')
display(법조치파산사건번호[~법조치파산사건번호.종국결과.str.contains('취하|기각|선고|종결|폐지|불허가|인용|채무자의 사망', na=True)])

# 사건명
print('(실사 후 전산수정필요,df는 일단 하면으로 해둠)상속재산이면 법조치구분을, 아니면 사건번호를 하면으로')
display(법조치파산사건번호[법조치파산사건번호.수정전사건번호.str.contains('하단', na=False) & (법조치파산사건번호.법조치구분!='상속재산파산')])

(df수정완료)회생법원으로 수정할 목록


,채무자키,새채무자키,이름키_,법조치키,대상자구분,대상자명,법조치구분,관할법원,사건번호,비고,법조치메모,종국결과,접수일,종료일,종국결과일,확정일,최신화일,이름키구분
0,20423483,n1035287,n1035287남기봉/도양물류4960,20651336,채무자,남기봉/도양물류4960,지급명령,서울남부지방법원,미접수 취하함,,,,2020-08-06,,,,,변경전채무자
559,20429888,n1048870,n1048870박명애,20642596,채무자,박명애,기타,부산지방법원,2011하면387,,,,2011-12-12,,,,,채무자
836,20429681,n1047736,n1047736케이엠에이치,20642461,채무자,케이엠에이치,지급명령,수원지방법원,2011하합1,,,,2011-02-07,,,,,채무자


(df수정필요)사건번호가 정규식에 부합하지 않음


,채무자키,새채무자키,이름키_,법조치키,대상자구분,대상자명,법조치구분,관할법원,사건번호,비고,법조치메모,종국결과,접수일,종료일,종국결과일,확정일,최신화일,이름키구분,수정사건번호
0,20423483,n1035287,n1035287남기봉/도양물류4960,20651336,채무자,남기봉/도양물류4960,지급명령,서울회생법원,미접수 취하함,,,,2020-08-06,,,,,변경전채무자,NaN


(df수정필요,실사!)상속재산파산 사건 하면 -> 하단 변경할 것


,채무자키,새채무자키,이름키_,법조치키,대상자구분,대상자명,법조치구분,관할법원,사건번호,당사포함여부,법조치메모,종국결과,접수일,종료일,종국결과일,확정일,최신화일,이름키구분,수정전사건번호
145,20410672,n1027209,n1027209신경섭,20908295,채무자,신경섭,상속재산파산,의정부지방법원,2024하단20418,당사포함,신청인\t 1. 장O원 \t망 신경섭 배우자\n 채권자\t 1. ㈜신한은행 \t...,,2024-03-20,,,,2025-03-31,채무자,2024하면20418
314,20426680,n1027737,n1027737홍상균,20904702,채무자,홍상균,상속재산파산,의정부지방법원,2020하단22023,당사포함,신청인 1. 이연숙 2022.03.07 \n채권자 7. 헬프자산관리대부,종결,2020-12-18,2022-03-04,2022-03-04,,2025-02-27,채무자,2020하면22023
470,20409906,n1025683,n1025683공정식,20908588,채무자,공정식,상속재산파산,창원지방법원,2023하단10132,당사포함,채권자 16. 솔림헬프자산관리대부 \n신청인 공용률 (망 공정식의 상속재산),,2023-02-15,,,,2025-03-31,채무자,2023하면10132


(df수정필요)목록에 없는 종국결과. 빈값은 아래서 엑셀출력


,채무자키,새채무자키,이름키_,법조치키,대상자구분,대상자명,법조치구분,관할법원,사건번호,당사포함여부,법조치메모,종국결과,접수일,종료일,종국결과일,확정일,최신화일,이름키구분,수정전사건번호
493,20414313,n1006598,n1006598이복자,20718547,채무자,이복자,파산,의정부지방법원,2010하면6638,,,확정,2013-07-29,,,,,NaN,NaN
552,20428458,n1029055,n1029055전용수,20716003,채무자,전용수,파산,강릉지원,2015하면361,,,불가,2015-12-30,,,,,채무자,NaN


(실사 후 전산수정필요,df는 일단 하면으로 해둠)상속재산이면 법조치구분을, 아니면 사건번호를 하면으로


,채무자키,새채무자키,이름키_,법조치키,대상자구분,대상자명,법조치구분,관할법원,사건번호,당사포함여부,법조치메모,종국결과,접수일,종료일,종국결과일,확정일,최신화일,이름키구분,수정전사건번호
11,20427310,n1029833,n1029833안상태,21433703,채무자,안상태,파산,인천지방법원,2025하면73,,인천지법 2025하단73 : 파산진행중,<NA>,2025-01-22,,,,2025-04-01,채무자,2025하단 73
48,20461804,n1043487,n1043487최석순,21435293,채무자,최석순,파산,창원지방법원,2024하면9193,포함,채권자 (주)솔림헬프자산관리대부,<NA>,2024-06-19,,,,2025-04-01,채무자,2024하단9193
146,20410672,n1027209,n1027209신경섭,20919515,채무자,신경섭,파산,의정부지방법원,2024하면20418,포함,신청인 1. 장O원 \n 채권자 1. ㈜신한은행 \n 채권자 2. ...,<NA>,2024-03-20,,,,2025-03-31,채무자,2024하단20418
281,20411439,n1005952,n1005952조유빈(변경전:조헌조),21432006,채무자,조유빈(변경전:조헌조),파산,창원지방법원,2024하면9043,포함,채권번호 1번 솔림헬프자산관리대부,<NA>,2024-02-06,,,,2025-03-31,변경전채무자,2024하단9043
282,20451102,n1018546,n1018546김훈오,21430529,채무자,김훈오,파산,부산회생법원,2024하면70517,포함,신청인 1. 김O오 \n 채권자 1. (주)솔림헬프자산관리대부 \n ...,폐지,2024-10-28,2025-02-14,2025-02-14,2025-03-01,2025-04-01,채무자,2024하단70517
283,20472091,n1031450,n1031450김영태,20921723,채무자,김영태,파산,울산지방법원,2024하면5269,,신청인 1. 김O태 \n 채권자 1. ㈜우리카드 \n 채권자 2. ...,<NA>,2024-10-21,,,,2025-04-01,채무자,2024하단5269
285,20429504,n1048025,n1048025진용희,21431825,보증인,진용희,파산,수원회생법원,2024하면14606,,신청인 1. 진O희 \n 채권자 1. (주)솔림헬프자산관리대부 \n ...,<NA>,2024-11-20,,,,,보증인,2024하단14606
286,20423967,n1058293,n1058293남화주,20921656,채무자,남화주,파산,서울회생법원,2024하면1285,포함,신청인 1. 남O주 \n 채권자 1. 신한카드(주) \n 채권자 2...,폐지,2024-09-30,2025-01-16,2025-01-16,2025-02-01,2025-04-01,채무자,2024하단1285
287,20427889,n1028017,n1028017김금성,21431824,보증인,김금성,파산,대전지방법원,2024하면10388,,신청인 1. 김O성 \n 채권자 1. 오O성 \n 채권자 2. 유한...,폐지,2024-04-01,2025-02-12,2025-02-12,2025-02-27,2025-03-31,보증인,2024하단10388
288,20419628,n1051818,n1051818남광현,20913969,,남광현,파산,인천지방법원,2024하면100852,,신청인 1. 이O란 \n 채권자 1. 주식회사 솔림헬프자산관리대부 \...,<NA>,2024-04-24,,,,2025-03-31,보증인,2024하단100852


#### 기본점검 수정/삭제

In [6]:
# 사건번호 없는 것 삭제
법조치파산사건번호 = 법조치파산사건번호.dropna(subset=['사건번호'])
법조치파산사건번호.loc[[493,552],"종국결과"] = ["인용","불허가"]

########## 사전정리인 경우, 사건번호 없는 것 외 나머지 기본점검사항은 전산만 수정 후 다시 내려받아 재작업하는 게 낫겠다.

#### 기본점검외 체크사항 엑셀출력
- pass가능. 아래서 df의 사건구분 등을 수정해서 여기서 출력해야함.. 중복사건 있으니 유의

In [7]:
# 법원사건번호 외 확인건 엑셀 출력 : 법조치구분, 대상자구분, 당사포함여부 미작성, 종국결과 없음(최신화), 
출력1 = 법조치파산사건번호[~법조치파산사건번호['법조치구분'].str.contains("파산", na=False)]
    # 대상자구분 : 대상자명을 같이 써줘야 대상자구분 수정됨.
출력2 = 법조치파산사건번호[((법조치파산사건번호['대상자구분']!=법조치파산사건번호["이름키구분"])) \
                & ~(법조치파산사건번호['대상자구분'].isin(['상속인','삭제보증인']) & 법조치파산사건번호["이름키구분"].isna())] # 빈칸이라도 대상자구분은 "", 이름키구분은 na라 다른 걸로 인식
출력3 = 법조치파산사건번호[~법조치파산사건번호['당사포함여부'].str.contains('포함|누락|기각|취하|불허가', na=False)]
출력4 = 법조치파산사건번호[법조치파산사건번호['종국결과'].isna()]

출력df = [출력1, 출력2, 출력3, 출력4]
출력파일명 = ['[DC]파산_법조치구분.xlsx', '[DC]파산_대상자구분.xlsx', '[DC]파산_당사포함여부.xlsx', '[DC]파산_종국결과확인.xlsx']

for df, filename in zip(출력df, 출력파일명):
    if not df.empty:
        print(f"{filename} → 저장 중 (행 수: {len(df)})")
        df.to_excel(filename, index=False)

[DC]파산_법조치구분.xlsx → 저장 중 (행 수: 8)
[DC]파산_대상자구분.xlsx → 저장 중 (행 수: 40)
[DC]파산_당사포함여부.xlsx → 저장 중 (행 수: 255)
[DC]파산_종국결과확인.xlsx → 저장 중 (행 수: 136)


#### 2. 채우기 및 충돌체크

In [8]:
# 빈값채우기 (빈 문자열은 NA로 먼저 바꿔줌 : 함수내부처리)
법조치파산사건번호 = fill_groupwise(df=법조치파산사건번호, column="당사포함여부")
법조치파산사건번호 = fill_groupwise(df=법조치파산사건번호, column="종국결과")
법조치파산사건번호 = fill_groupwise(df=법조치파산사건번호, column="접수일")
법조치파산사건번호 = fill_groupwise(df=법조치파산사건번호, column="종료일")
법조치파산사건번호 = fill_groupwise(df=법조치파산사건번호, column="종국결과일")

In [9]:
temp = 법조치파산사건번호.replace("", pd.NA)
temp = temp.reset_index() # index 칼럼에 기존 index 저장
충돌체크용 = temp.set_index(["새채무자키", "사건번호"])[["index", "채무자키","관할법원", "대상자명", "대상자구분", "당사포함여부", "종국결과", "종국결과일", "최신화일"]]

grouped = 충돌체크용.groupby(["새채무자키", "사건번호"])

관할법원충돌 = grouped["관할법원"].nunique(dropna=True)
대상자명충돌 = grouped["대상자명"].nunique(dropna=True)
대상자구분충돌 = grouped["대상자구분"].nunique(dropna=True)
당사포함여부충돌 = grouped["당사포함여부"].nunique(dropna=True)   # dropna=True: NaN 제외하고 셈
종국결과충돌 = grouped["종국결과"].nunique(dropna=True)
종국결과일충돌 = grouped["종국결과일"].nunique(dropna=True)

for 컬럼명, 충돌 in [
    ("관할법원", 관할법원충돌),
    ("대상자명", 대상자명충돌),
    ("대상자구분", 대상자구분충돌),
    ("당사포함여부", 당사포함여부충돌),
    ("종국결과", 종국결과충돌),
    ("종국결과일충돌", 종국결과일충돌),
]:
    
    print(f'{컬럼명} 충돌 ----')
    display(충돌체크용.loc[충돌[충돌 > 1].index])

관할법원 충돌 ----


index      채무자키    관할법원 대상자명 대상자구분 당사포함여부 종국결과  \
새채무자키    사건번호                                                         
n1051774 2014하면2332    805  20420633  서울회생법원  임호범   채무자     포함   인용   
         2014하면2332    806  20420633  수원회생법원  임호범   채무자     포함   인용   

                          종국결과일        최신화일  
새채무자키    사건번호                                
n1051774 2014하면2332  2015-07-15  2025-04-01  
         2014하면2332  2015-07-15  2025-04-01

대상자명 충돌 ----


,,index,채무자키,관할법원,대상자명,대상자구분,당사포함여부,종국결과,종국결과일,최신화일
새채무자키,사건번호,,,,,,,,,


대상자구분 충돌 ----


...

,,index,채무자키,관할법원,대상자명,대상자구분,당사포함여부,종국결과,종국결과일,최신화일
새채무자키,사건번호,,,,,,,,,


종국결과 충돌 ----


,,index,채무자키,관할법원,대상자명,대상자구분,당사포함여부,종국결과,종국결과일,최신화일
새채무자키,사건번호,,,,,,,,,


종국결과일충돌 충돌 ----


,,index,채무자키,관할법원,대상자명,대상자구분,당사포함여부,종국결과,종국결과일,최신화일
새채무자키,사건번호,,,,,,,,,


#### 충돌 수정

In [10]:
# 문제된 것 수정
법조치파산사건번호.loc[805, "관할법원"] = "수원회생법원"

#### 대상자구분 수정 및 채우기

In [11]:
# 법조치 대상자구분 종류 : 채무자, 보증인, 상속인, 삭제보증인
# 이름키구분 종류 : 채무자, 보증인, 변경전채무자(이건 법조치 수정할 것)

# 1) 대상자구분 빈값 -> 이름키구분
법조치파산사건번호['대상자구분'].replace("", pd.NA, inplace=True)
법조치파산사건번호['대상자구분'] = 법조치파산사건번호['대상자구분'].fillna(법조치파산사건번호["이름키구분"])

# 2) 대상자구분 채무자, 이름키구분 보증인 -> 이름키구분
cond = (법조치파산사건번호['대상자구분']=="채무자") & (법조치파산사건번호["이름키구분"]=="보증인")
법조치파산사건번호.loc[cond, "대상자구분"] = "보증인"

print('법조치 대상자구분 종류 :',법조치파산사건번호['대상자구분'].unique())

# 변경전채무자 목록
##########################################
if company == "솔림" : 
    확인완료법조치키 = ["21432006","20907145","20910524"]
else : 
    확인완료법조치키 = []
##########################################
print('')
print('변경전채무자 사건조회되는 이름으로 전산 수정하라(변경전으로 완료됐으면 변경전으로 조회됨)')
display(법조치파산사건번호[(법조치파산사건번호['대상자구분']=="변경전채무자") & ~법조치파산사건번호["법조치키"].isin(확인완료법조치키)])

print('대상자구분 여전히 빈값인 것 수정')
대상자구분빈값ind = 법조치파산사건번호[법조치파산사건번호.대상자구분.isna()].index
display(법조치파산사건번호.loc[대상자구분빈값ind])

법조치 대상자구분 종류 : ['보증인' '채무자' nan]

변경전채무자 사건조회되는 이름으로 전산 수정하라(변경전으로 완료됐으면 변경전으로 조회됨)


,채무자키,새채무자키,이름키_,법조치키,대상자구분,대상자명,법조치구분,관할법원,사건번호,당사포함여부,법조치메모,종국결과,접수일,종료일,종국결과일,확정일,최신화일,이름키구분,수정전사건번호


대상자구분 여전히 빈값인 것 수정


,채무자키,새채무자키,이름키_,법조치키,대상자구분,대상자명,법조치구분,관할법원,사건번호,당사포함여부,법조치메모,종국결과,접수일,종료일,종국결과일,확정일,최신화일,이름키구분,수정전사건번호
284,20427573,n1028117,n1028117김주봉,20912457,NaN,김주봉,상속재산파산,부산회생법원,2024하단243,포함,신청인 1. 김O봉 \n 채권자 1. 신한카드(주) \n 채권자 2...,폐지,2024-04-04,2025-02-14,2025-02-14,2025-03-01,2025-04-01,NaN,NaN
871,20414440,n1006250,n1006250이은경,20739122,NaN,이은경,파산,대구지방법원,2010하면5726,포함,3. 우리에이치비제일차유동화전문 유한회사,인용,2010-08-20,2012-01-18,<NA>,,,NaN,NaN


In [12]:
# 대상자구분 빈값 수정
법조치파산사건번호.loc[대상자구분빈값ind, '대상자구분'] = ["상속인", "삭제보증인"]

In [13]:
# 충돌여부 재확인
temp = 법조치파산사건번호.replace("", pd.NA)
temp = temp.reset_index() # index 칼럼에 기존 index 저장
충돌체크용 = temp.set_index(["새채무자키", "사건번호"])[["index", "채무자키","관할법원", "대상자명", "대상자구분", "당사포함여부", "종국결과", "종국결과일", "최신화일"]]

grouped = 충돌체크용.groupby(["새채무자키", "사건번호"])

대상자구분충돌 = grouped["대상자구분"].nunique(dropna=True)
   
print(f'{"대상자구분"} 충돌 ----')
display(충돌체크용.loc[충돌[충돌 > 1].index])

대상자구분 충돌 ----


,,index,채무자키,관할법원,대상자명,대상자구분,당사포함여부,종국결과,종국결과일,최신화일
새채무자키,사건번호,,,,,,,,,


In [14]:
# 충돌 문제된 거 수정


#### 3. 정렬 및 중복제거

In [15]:
# 정렬 및 중복건제거                              
# 원래 정렬기준은 ["종국결과", "종국결과일", "당사포함여부", "새채무자키"] 였으나, 빈값채우고 충돌여부 모두 체크했으므로 최신화일만 고려(최신화일 다른 건 많음)
print('중복제거 전', len(법조치파산사건번호))
법조치파산사건번호 = 법조치파산사건번호.sort_values(by=["최신화일", "새채무자키", "사건번호", "채무자키", "대상자구분"], ascending=[False, True, True, True, False]) 
법조치파산사건번호 = 법조치파산사건번호.drop_duplicates(subset=["새채무자키","사건번호"]) # 관할법원에 오류가 있을 수 있으므로 관할법원+사건번호 대신 새채무자키 사용
print('중복제거 후', len(법조치파산사건번호))

중복제거 전 947
중복제거 후 795


### 채무자,보증인 파산사건번호 추출
- 일부 종결계좌가 있더라도 파산사건번호 추출이 목적이므로 모든 채권 종결아니면 남긴다

#### 1. 기본점검 : 종결, 법원사건번호, 당사포함여부

In [16]:
# 채무자새창에서 파산 사건번호 추출
                # 법조치사건번호는 법조치새창에서 최근 등록한 법조치를 불러오는 것이므로 여기서 다룰 필요 없음
미종결debt = debt[debt['새채무자키'].isin(미종결새채무자키)][["채무자키","새채무자키","이름키_","주민번호인","성명","회생관할법원","회생사건번호","회생사건결과",
                                              "관리자관할법원","관리자사건번호","관리자사건결과","파산관할법원","파산사건번호","파산사건결과","회생메모"]]

기본cols = ["채무자키","새채무자키","주민번호인","성명","회생메모", "이름키_"]
temp1 = 미종결debt.query('회생사건번호.str.contains("하")')[기본cols + ["회생관할법원","회생사건번호","회생사건결과"]]
temp1.rename(columns={'회생관할법원':'관할법원','회생사건번호':'사건번호',"회생사건결과":'종국결과'}, inplace=True)
temp2 = 미종결debt.query('관리자사건번호.str.contains("하")')[기본cols + ["관리자관할법원","관리자사건번호","관리자사건결과"]]
temp2.rename(columns={'관리자관할법원':'관할법원','관리자사건번호':'사건번호',"관리자사건결과":'종국결과'}, inplace=True)
temp3 = 미종결debt.query('파산사건번호.str.contains("하")')[기본cols + ["파산관할법원","파산사건번호","파산사건결과"]]
temp3.rename(columns={'파산관할법원':'관할법원','파산사건번호':'사건번호',"파산사건결과":'종국결과'}, inplace=True)

채무자파산사건번호 = pd.concat([temp1, temp2, temp3])
채무자파산사건번호 = 채무자파산사건번호.rename(columns={'회생메모':'당사포함여부', "성명" : "대상자명"})
    # 종국결과열 정리
채무자파산사건번호["종국결과일"] = 채무자파산사건번호.종국결과.str.extract(r'(\d{4}-\d{2}-\d{2})')
채무자파산사건번호["종국결과"] = 채무자파산사건번호["종국결과"].str.extract(r'([가-힣]+)')
    # 대상자구분열
채무자파산사건번호["대상자구분"] = "채무자"



# 보증인새창에서 파산사건번호추출
    # 보증인새창에 새채무자키 입히기
grt = grt.merge(debt[["채무자키", "새채무자키"]])
    # 모든 채무 면탈된 보증인 제외
미종결grt = grt[grt["주민번호인"].isin(미종결보증인주민번호)][["채무자키","새채무자키","이름키_","주민번호인","보증인성명인","관할법원","사건번호", "비고"]]
    # 계좌 미종결건 남기기
미종결grt = 미종결grt[미종결grt['새채무자키'].isin(미종결새채무자키)]

보증인파산사건번호 = 미종결grt.query('사건번호.str.contains("하")')
보증인파산사건번호 = 보증인파산사건번호.rename(columns={"비고":"당사포함여부", "보증인성명인":"대상자명"})
    # 대상자구분열
보증인파산사건번호["대상자구분"] = "보증인"



# 병합(채무자, 보증인 둘다 있는 경우, 보증인행을 살린다. 대신 포함여부는 채무자에만 있을 확률이 높기 때문에 삭제전 포함여부를 모든 행에 채운다.)
    # 1. 병합 및 전처리
파산사건번호 = pd.concat([채무자파산사건번호, 보증인파산사건번호], ignore_index=True)
병합직후개수 = len(파산사건번호)
        # 당사포함여부 정리
conds = [파산사건번호["당사포함여부"].str.contains('포함', na=False), 파산사건번호["당사포함여부"].str.contains('누락', na=False)]
values = ["포함", "누락"]
파산사건번호["당사포함여부"] = np.select(conds, values, default="")
        # 이름키구분열(이름이 현재 전산에 있는지 여부) [채무자, 보증인, 빈칸] -> 빈칸이면 해당 채무자키에 해당 이름을 가진 채무자가 없음.(개명/오타가능성 있으나 이건 무시하자..)
파산사건번호 = 파산사건번호.merge(전체이름키, on='이름키_', how='left') # 전체이름키 : 이름키, 이름키구분 열로 구성됨

        # 법원사건번호 수정(df는 바로 수정함)
            # 법원
print('(df수정완료)회생법원으로 수정할 목록. 전산 즉시 수작업')
display(파산사건번호[파산사건번호.관할법원.str.contains('수원지방|부산지방|서울(?!.*회생)')])
파산사건번호["관할법원"] = 파산사건번호["관할법원"].replace({'수원지방법원':'수원회생법원','부산지방법원':'부산회생법원',r'서울.*지방법원':'서울회생법원'}, regex=True)

            # 사건번호(쓸데없는 말 및 공백 제거, 하단은 하면으로)
파산사건번호["수정사건번호"] = 파산사건번호["사건번호"].str.extract(파산사건정규식)
파산사건번호["수정사건번호"] = 파산사건번호["수정사건번호"].replace({"하단":"하면", r"\s+":""}, regex=True)
            # 칼럼명 변경
파산사건번호 = 파산사건번호.rename(columns={'수정사건번호':'사건번호','사건번호':'수정전사건번호'})
파산사건번호 = functions.swap_columns(파산사건번호, '수정전사건번호', '사건번호')
            # 보기 편하게 수정전후가 같은 겉은 수정전을 빈값으로
파산사건번호["수정전사건번호"] = 파산사건번호["수정전사건번호"].where(파산사건번호["수정전사건번호"]!=파산사건번호["사건번호"])
            
            
# [dc]사건번호가 정규식에 부합하지 않음
print("(df수정필요)사건번호가 정규식에 부합하지 않음. 전산 즉시 수작업")
display(파산사건번호[파산사건번호.사건번호.isna()]) # 수정 및 법조치 등록은 수작업으로



# 종국결과 : 
# 하단 : (신청)취하, 기각 / 파산선고, 파산선고및동시폐지, 종결, 폐지(하면으로 넘어감)
# 하면 : (신청)취하, 기각 / 불허가 / 인용 ** 채무자의 : 채무자의 사망으로 종결
    # 채무자의 -> 채무자의 사망으로 종료
파산사건번호.loc[파산사건번호.종국결과.str.contains('채무자의', na=False), '종국결과'] = "채무자의 사망으로 종료"
    # 종국결과가 정해진 값이 아닌 것
print('(df수정필요)목록에 없는 종국결과. 빈값은 아래서 엑셀출력')
display(파산사건번호[~파산사건번호.종국결과.str.contains('취하|기각|선고|종결|폐지|불허가|인용|채무자의 사망', na=True)])

(df수정완료)회생법원으로 수정할 목록. 전산 즉시 수작업


,채무자키,새채무자키,주민번호인,대상자명,당사포함여부,이름키_,관할법원,사건번호,종국결과,종국결과일,대상자구분,이름키구분
707,20414295,n1006471,600826-2806419,김인숙,,n1006471김인숙,수원지방법원,2021하단2326,NaN,NaN,보증인,보증인
729,20418556,n1002213,610910-1475811,조경제,,n1002213조경제,수원지방법원,2022하단12009,NaN,NaN,보증인,보증인
747,20426808,n1027508,741111-1659514,이왕일,,n1027508이왕일,수원지방법원,2017하면100185,NaN,NaN,보증인,보증인
754,20428965,n1048733,530610-2468620,김숙자,,n1048733김숙자,수원지방법원,2021하단10026,NaN,NaN,보증인,보증인
756,20429431,n1047752,600215-2090915,차신화,,n1047752차신화,수원지방법원,2021하단2284,NaN,NaN,보증인,보증인
757,20429437,n1047806,560820-1351163,권오상,,n1047806권오상,수원지방법원,2022하단11584,NaN,NaN,보증인,보증인
767,20429678,n1047783,600627-1023312,노창우,,n1047783노창우,수원지방법원,2021하단2006,NaN,NaN,보증인,보증인
776,20552753,n1058606,560903-1079817,이갑수,,n1058606이갑수,수원지방법원,2014하면2859,NaN,NaN,보증인,보증인


(df수정필요)사건번호가 정규식에 부합하지 않음. 전산 즉시 수작업


,채무자키,새채무자키,주민번호인,대상자명,당사포함여부,이름키_,관할법원,사건번호,종국결과,종국결과일,대상자구분,이름키구분,수정전사건번호
189,20451102,n1018546,770301-1904611,김훈오,,n1018546김훈오,부산회생법원,NaN,NaN,NaN,채무자,채무자,224하면70517
190,20451103,n1018546,770301-1904611,김훈오,,n1018546김훈오,부산회생법원,NaN,NaN,NaN,채무자,채무자,224하면70517
191,20451104,n1018546,770301-1904611,김훈오,,n1018546김훈오,부산회생법원,NaN,NaN,NaN,채무자,채무자,224하면70517
192,20451105,n1018546,770301-1904611,김훈오,,n1018546김훈오,부산회생법원,NaN,NaN,NaN,채무자,채무자,224하면70517
193,20451106,n1018546,770301-1904611,김훈오,,n1018546김훈오,부산회생법원,NaN,NaN,NaN,채무자,채무자,224하면70517
194,20451107,n1018546,770301-1904611,김훈오,,n1018546김훈오,부산회생법원,NaN,NaN,NaN,채무자,채무자,224하면70517


(df수정필요)목록에 없는 종국결과. 빈값은 아래서 엑셀출력


,채무자키,새채무자키,주민번호인,대상자명,당사포함여부,이름키_,관할법원,사건번호,종국결과,종국결과일,대상자구분,이름키구분,수정전사건번호


#### 기본작업 후 수정(사건번호)

In [17]:
# 사건번호 : 정규식에 부합하지 않는 건 즉시 수정/삭제 (법조치에 올바른 거 있거나, 법조치 등록 후 재작업 할 거라면 삭제)
파산사건번호 = 파산사건번호.dropna(subset=['사건번호'])
# # 종국결과
# 파산사건번호 = 파산사건번호.drop(index=[493, 552])

#### 2.채우기, 병합우선순위(보증), 충돌체크

In [18]:
    # 2. 동일사건 빈칸 채우기
파산사건번호 = fill_groupwise(파산사건번호, "당사포함여부")
파산사건번호 = fill_groupwise(파산사건번호, "종국결과")
파산사건번호 = fill_groupwise(파산사건번호, "종국결과일")

    # 3. 보증인에 있는 조합이 기존 채무자에도 있다면 → 채무자 쪽 행 제거
        # 보증인 쪽에 있는 (새채무자키, 사건번호) 키
중복키 = (
    파산사건번호[["새채무자키", "사건번호", "대상자구분"]]
    .drop_duplicates()
    .pivot_table(index=["새채무자키", "사건번호"], columns="대상자구분", aggfunc="size", fill_value=0)
)

        # '채무자'와 '보증인' 둘 다 있는 경우만 추출
양쪽다있는키 = 중복키[(중복키.get("채무자",0) > 0) & (중복키.get("보증인",0) > 0)].index

        # 해당 키이면서 대상자구분이 '채무자'인 행만 제거
파산사건번호 = 파산사건번호[
    ~((파산사건번호.set_index(["새채무자키", "사건번호"]).index.isin(양쪽다있는키)) & (파산사건번호["대상자구분"] == "채무자"))
].reset_index(drop=True)

print(f'채무자건수:{len(채무자파산사건번호)}, 보증인건수{len(보증인파산사건번호)}, \
단순합(병합직후):{병합직후개수}, 양자중복건중 차주건 삭제후:{len(파산사건번호)}, 삭제건수:{병합직후개수-len(파산사건번호)}')
print('(채무자) 삭제 목록 :', 양쪽다있는키.values.tolist())

채무자건수:691, 보증인건수86, 단순합(병합직후):777, 양자중복건중 차주건 삭제후:765, 삭제건수:12
(채무자) 삭제 목록 : [('n1044643', '2018하면1452'), ('n1047783', '2021하면2006'), ('n1047893', '2022하면11970'), ('n1049155', '2017하면649')]


In [19]:
temp = 파산사건번호.replace("", pd.NA)
temp = temp.reset_index() # index 칼럼에 기존 index 저장  
충돌체크용 = temp.set_index(["새채무자키", "사건번호"])[["index", "채무자키","관할법원", "대상자명", "대상자구분", "당사포함여부", "종국결과", "종국결과일"]]

grouped = 충돌체크용.groupby(["새채무자키", "사건번호"])

관할법원충돌 = grouped["관할법원"].nunique(dropna=True)
대상자명충돌 = grouped["대상자명"].nunique(dropna=True)
대상자구분충돌 = grouped["대상자구분"].nunique(dropna=True)
당사포함여부충돌 = grouped["당사포함여부"].nunique(dropna=True)  
종국결과충돌 = grouped["종국결과"].nunique(dropna=True)


for 컬럼명, 충돌 in [
    ("관할법원", 관할법원충돌),
    ("대상자명", 대상자명충돌),
    ("대상자구분", 대상자구분충돌),
    ("당사포함여부", 당사포함여부충돌),
    ("종국결과", 종국결과충돌)
]:
    
    print(f'{컬럼명} 충돌 ----')
    display(충돌체크용.loc[충돌[충돌 > 1].index])
    
### 보증인 우선으로 중복 차주건 삭제했으니 충돌건 아마 없을 것이다.

관할법원 충돌 ----


,,index,채무자키,관할법원,대상자명,대상자구분,당사포함여부,종국결과,종국결과일
새채무자키,사건번호,,,,,,,,


대상자명 충돌 ----


,,index,채무자키,관할법원,대상자명,대상자구분,당사포함여부,종국결과,종국결과일
새채무자키,사건번호,,,,,,,,


대상자구분 충돌 ----


,,index,채무자키,관할법원,대상자명,대상자구분,당사포함여부,종국결과,종국결과일
새채무자키,사건번호,,,,,,,,


당사포함여부 충돌 ----


,,index,채무자키,관할법원,대상자명,대상자구분,당사포함여부,종국결과,종국결과일
새채무자키,사건번호,,,,,,,,


종국결과 충돌 ----


,,index,채무자키,관할법원,대상자명,대상자구분,당사포함여부,종국결과,종국결과일
새채무자키,사건번호,,,,,,,,


#### 충돌수정, 정렬 및 중복제거

In [20]:
# # # 문제된 것 수정
# # 파산사건번호.loc[126374, "관할법원"] = "수원회생법원"
# 파산사건번호 = 파산사건번호.drop(index=[585,586,587,613,270,646])

#### 3. 정렬 및 중복제거

In [21]:
# 정렬 및 중복제거
    # (종국결과 등 채우기 했으므로) 키값기준으로 정렬
print('중복제거전', len(파산사건번호))
파산사건번호 = 파산사건번호.sort_values(by=["새채무자키","사건번호","채무자키","대상자구분"], ascending=[True, True, True, False])
    # 중복제거
파산사건번호 = 파산사건번호.drop_duplicates(subset=["새채무자키","사건번호"])
print('병합 및 중복제거후',len(파산사건번호))

중복제거전 765
병합 및 중복제거후 733


#### 법조치 미등록건

In [22]:
미등록파산사건번호 = pd.merge(파산사건번호, 법조치파산사건번호[["새채무자키","사건번호"]], how='left', indicator=True)
미등록파산사건번호 = 미등록파산사건번호[미등록파산사건번호['_merge']== 'left_only']
if not 미등록파산사건번호.empty :
    print(f'미등록 파산사건 {len(미등록파산사건번호)}건 엑셀출력')
    미등록파산사건번호.to_excel('미등록파산사건번호.xlsx', index=False)
else :
    print(f'미등록 파산사건 없음')

미등록 파산사건 78건 엑셀출력


### 파산 기준데이터
- 최신화실패이든 아니든 종국결과 없는 것은 파산(진행확정)으로ㅡ

#### 1. 병합

In [23]:
# (기본점검은 위에서 다 했으므로 다시 할 필요 없음)
공통cols = ["채무자키","새채무자키","대상자명", "대상자구분", "관할법원","사건번호","당사포함여부", "종국결과", "종국결과일","이름키_","이름키구분", "수정전사건번호"]
법조치cols = ['접수일','종료일', '확정일','최신화일','법조치키', "법조치구분"]

파산 = pd.concat([법조치파산사건번호[공통cols+법조치cols], 파산사건번호[공통cols]])
print('단순병합 후 건수 :',len(파산))
# 사건번호 없는 것은 사전에 삭제/수정했음

단순병합 후 건수 : 1528


#### 2. 채우기, 충돌체크 후 삭제

##### 채우기, 기본충돌체크

In [24]:
# 빈값채우기
파산 = fill_groupwise(df=파산, column="당사포함여부")
파산 = fill_groupwise(df=파산, column="종국결과")
파산 = fill_groupwise(df=파산, column="접수일")
파산 = fill_groupwise(df=파산, column="종료일")
파산 = fill_groupwise(df=파산, column="종국결과일")

In [25]:
temp = 파산.replace("", pd.NA)
temp = temp.reset_index() # index 칼럼에 기존 index 저장
충돌체크용 = temp.set_index(["새채무자키", "사건번호"])[["index", "채무자키","관할법원", "대상자명", "대상자구분", "당사포함여부", "종국결과", "종국결과일", "최신화일","이름키구분"]]

grouped = 충돌체크용.groupby(["새채무자키", "사건번호"])

관할법원충돌 = grouped["관할법원"].nunique(dropna=True)
당사포함여부충돌 = grouped["당사포함여부"].nunique(dropna=True)  
종국결과충돌 = grouped["종국결과"].nunique(dropna=True)
종국결과일충돌 = grouped["종국결과일"].nunique(dropna=True)


for 컬럼명, 충돌 in [
    ("관할법원", 관할법원충돌),
    ("당사포함여부", 당사포함여부충돌),
    ("종국결과", 종국결과충돌),
    ("종국결과일", 종국결과일충돌),
]:
    
    print(f'{컬럼명} 충돌 ----')
    display(충돌체크용.loc[충돌[충돌 > 1].index])

관할법원 충돌 ----


,,index,채무자키,관할법원,대상자명,대상자구분,당사포함여부,종국결과,종국결과일,최신화일,이름키구분
새채무자키,사건번호,,,,,,,,,,


당사포함여부 충돌 ----


,,index,채무자키,관할법원,대상자명,대상자구분,당사포함여부,종국결과,종국결과일,최신화일,이름키구분
새채무자키,사건번호,,,,,,,,,,


종국결과 충돌 ----


,,index,채무자키,관할법원,대상자명,대상자구분,당사포함여부,종국결과,종국결과일,최신화일,이름키구분
새채무자키,사건번호,,,,,,,,,,


종국결과일 충돌 ----


,,index,채무자키,관할법원,대상자명,대상자구분,당사포함여부,종국결과,종국결과일,최신화일,이름키구분
새채무자키,사건번호,,,,,,,,,,


In [26]:
# 대상자 구분 사전 점검 및 수정정
print('대상자구분 종류 :',파산.대상자구분.unique()) # 아직 상속인 등 업데이트 한 파일이 아니라서 채무자와 보증인밖에 없네...
print('이름키구분 종류 :',파산.이름키구분.unique())

파산.대상자구분.fillna("", inplace=True)
파산.이름키구분.fillna("", inplace=True)

# 대상자구분과 이름키구분이 다른 거 먼저 확인 
print('')
print('(df수정필요, 실사!) 대상자구분과 이름키구분이 다름')
cond = (
    (파산.대상자구분 != 파산.이름키구분) &
    ~((파산.대상자구분=="상속인") & ((파산.이름키구분=="보증인")|(파산.이름키구분==""))) &       # 법원 대상자구분 유지1
    ~((파산.대상자구분=="채무자") & (파산.이름키구분=="변경전채무자")) &  # 법원 대상자구분 유지2
    ~((파산.대상자구분=="삭제보증인") & (파산.이름키구분==""))          # 법원 대상자구분 유지3
    )
파산.loc[cond]

대상자구분 종류 : ['채무자' '보증인' '상속인' '삭제보증인']
이름키구분 종류 : ['채무자' '보증인' '변경전채무자' nan]

(df수정필요, 실사!) 대상자구분과 이름키구분이 다름


,채무자키,새채무자키,대상자명,대상자구분,관할법원,사건번호,당사포함여부,종국결과,종국결과일,이름키_,이름키구분,수정전사건번호,접수일,종료일,확정일,최신화일,법조치키,법조치구분
788,20412765,n1053887,김경수,보증인,의정부지방법원,2017하면2692,포함,인용,2018-12-21,n1053887김경수,,NaN,2017-12-19,2018-12-21,2019-01-15,2025-04-01,20909348,파산
787,20412794,n1053907,최창우,보증인,수원회생법원,2017하면3375,포함,인용,2018-04-26,n1053907최창우,,NaN,2017-09-13,2018-04-26,2018-05-11,2025-04-01,20909347,파산
311,20427836,n1028021,윤동천,채무자,서울회생법원,2019하면103687,<NA>,인용,2020-04-17,n1028021윤동천,,NaN,2019-10-07,2020-04-17,,2022-05-23,20664776,파산
807,20412808,n1053786,이병학,채무자,서울회생법원,2006하면41751,<NA>,인용,<NA>,n1053786이병학,,NaN,2006-01-01,<NA>,,2022-05-22,20733464,파산
605,20412756,n1053924,조유선,채무자,청주지방법원,2007하면1425,<NA>,인용,2008-01-04,n1053924조유선,,NaN,2007-01-01,2008-01-04,,2022-05-22,20648615,파산
367,20413420,n1054841,박동석,채무자,대구지방법원,2010하면6217,<NA>,파산선고,2011-12-15,n1054841박동석,,NaN,2010-01-01,2011-12-15,,2022-05-22,20658375,파산
366,20413420,n1054841,김명수,채무자,대구지방법원,2014하면4340,<NA>,폐지,2015-11-30,n1054841김명수,,NaN,2014-01-01,2015-11-30,,2022-05-22,20658372,파산
697,20420743,n1055794,김종진,채무자,창원지방법원,2011하면101,<NA>,인용,2013-01-10,n1055794김종진,,NaN,<NA>,2013-01-10,,2022-05-22,20657878,파산
493,20414313,n1006598,이복자,채무자,의정부지방법원,2010하면6638,<NA>,인용,<NA>,n1006598이복자,,NaN,2013-07-29,<NA>,,,20718547,파산


In [27]:
# 전산수정 및 df수정 : 
# 실사 필요. 셋중에 하나일 것이다(대부분 삭제보증인) [삭제보증인|변경전채무자|변경전보증인], (주) 처리한 후로 현재까지 채무자인 건은 없었음
파산.loc[cond, "대상자구분"] = "삭제보증인"

##### 충돌삭제 : 대상자구분, 대상자명

In [28]:
# 작업 전 확인
파산.대상자구분.value_counts() # 대상자명이 변경전 채무자이름으로 등록되어있더라도 대상자구분값은 채무자로 둔다. '변경전채무자'는 이름키구분에만 있는 값

대상자구분
채무자      1323
보증인       194
삭제보증인      10
상속인         1
Name: count, dtype: int64

In [29]:
# 종국결과일까지 충돌을 처리 한 후 작업 ---------------------------------
# (전산수정불요)주로 채무자상세에 보증인 파산사건 적어서 일어나는 충돌. 굳이 삭제필요 없음
삭제전건수 = len(파산)
조회용파산 = 파산.set_index(["새채무자키","사건번호"]).sort_index() # group객체로는 여러 인덱스를 한번에 조회할 수 없음

# 1) 대상자구분 : [보증인|삭제보증인|상속인]이 채무자에 우선(법조치 대상자구분은 '채무자'가 기본값인데 이로 인한 충돌 99.999%)
    # 대상자구분 고유수 확인
파산["대상자구분_고유수"] = (
    파산
    .groupby(["새채무자키", "사건번호"])["대상자구분"]
    .transform(lambda x: x.nunique(dropna=True))
)
    # '대상자구분_고유수'가 2개이고, '채무자'인 경우만 제거
삭제conds = ((파산["대상자구분_고유수"] == 2) & (파산["대상자구분"] == "채무자")) 
대상자구분충돌ind = 파산[삭제conds].set_index(["새채무자키","사건번호"]).index

    # 파산 재정의 : 제외처리
파산 = 파산[~삭제conds].drop(columns="대상자구분_고유수").reset_index(drop=True)
대상자구분삭제후건수 = len(파산)


# 2) 대상자명(이름키구분 사용) : 대상자구분을 먼저 처리했을 것이므로 대상자구분은 같을 것. 이름키구분 : '채무자'가 '변경전채무자'에 우선
파산["이름키구분_고유수"] = (
    파산
    .groupby(["새채무자키", "사건번호"])["이름키구분"]
    .transform(lambda x: x.nunique(dropna=True))
)
    # '대상자구분_고유수'가 2개이고, '채무자'인 경우만 제거
삭제conds = ((파산["이름키구분_고유수"] == 2) & (파산["이름키구분"] == "변경전채무자"))

이름키구분충돌ind = 파산[삭제conds].set_index(["새채무자키","사건번호"]).index

    # 파산 재정의 : 제외처리
파산 = 파산[~삭제conds].drop(columns="이름키구분_고유수").reset_index(drop=True)
이름키구분삭제후건수 = len(파산)

print(f'삭제전건수:{삭제전건수}, 대상자구분삭제후건수:{대상자구분삭제후건수}, 이름키구분삭제후건수:{이름키구분삭제후건수}')
print("")
print('대상자구분 삭제건 확인')
display(조회용파산[조회용파산.index.isin(대상자구분충돌ind)][["채무자키","대상자구분","대상자명","이름키구분"]])
print('이름키구분 삭제건 확인')
display(조회용파산[조회용파산.index.isin(이름키구분충돌ind)][["채무자키","대상자구분","대상자명","이름키구분"]])

삭제전건수:1528, 대상자구분삭제후건수:1514, 이름키구분삭제후건수:1511

대상자구분 삭제건 확인


채무자키  대상자구분        대상자명 이름키구분
새채무자키    사건번호                                           
n1001198 2023하면13905   20411446    보증인         박원희   보증인
         2023하면13905   20411446    채무자     ㈜한양트레이딩   채무자
n1006037 2023하면105526  20414316    보증인         오세용   보증인
         2023하면105526  20414316    채무자  ㈜포시즌엔터프라이즈   채무자
n1006200 2021하면50798   20419379    보증인         이득수   보증인
         2021하면50798   20419379    채무자       ㈜대우산업   채무자
n1044606 2024하면104185  20494401    보증인         박성철   보증인
         2024하면104185  20494401    채무자    지피정보시스템㈜   채무자
n1047857 2024하면112     20429473    보증인         고득곤   보증인
         2024하면112     20429473    채무자     (유)신영목재   채무자
n1048018 2023하면20996   20429555    보증인         김근배   보증인
         2023하면20996   20429555    채무자      한국코아제약   채무자
n1048654 2024하면14247   20429852    보증인         김국종   보증인
         2024하면14247   20429852    채무자    ㈜비케이텍스타일   채무자
n1051757 2023하면21654   20419668    보증인         박영길   보증인
         2023하면21654   20419668    채무자       ㈜영신산업   채무자
n1053741 2021하면10512   20419708    보증인         이종화   보증인
         2021하면10512   20419708    채무자       ㈜우일화학   채무자
n1053887 2017하면2692    20412765  삭제보증인         김경수      
         2017하면2692    20412765    채무자       주오건설㈜   채무자
n1054293 2022하면100345  20412909    보증인         김상대   보증인
         2022하면100345  20412909    채무자     광신로프산업㈜   채무자
n1054329 2024하면11894   20412837    보증인         전순희   보증인
         2024하면11894   20412837    채무자         ㈜영풍   채무자
n1055815 2024하면77      20421583    보증인         변영환   보증인
         2024하면77      20421583    채무자         유은영   채무자
n1058595 2021하면3599    20552697    보증인         진상숙   보증인
         2021하면3599    20552697    채무자         유한민   채무자

이름키구분 삭제건 확인


채무자키 대상자구분          대상자명   이름키구분
새채무자키    사건번호                                            
n1001848 2017하면356   20411254   채무자           홍미자  변경전채무자
         2017하면356   20411254   채무자           홍지안     채무자
n1002150 2013하면771   20415084   채무자           김향숙  변경전채무자
         2013하면771   20415084   채무자           김정윤     채무자
n1005952 2024하면9043  20411439   채무자  조유빈(변경전:조헌조)  변경전채무자
         2024하면9043  20411439   채무자           조유빈     채무자

In [30]:
temp = 파산.copy()
temp = temp.reset_index() # index 칼럼에 기존 index 저장
충돌체크용 = temp.set_index(["새채무자키", "사건번호"])[["index", "채무자키","관할법원", "대상자명", "대상자구분", "당사포함여부", "종국결과", "종국결과일", "최신화일","이름키구분"]]

grouped = 충돌체크용.groupby(["새채무자키", "사건번호"])


대상자명충돌 = grouped["대상자명"].nunique(dropna=True)
대상자구분충돌 = grouped["대상자구분"].nunique(dropna=True)

for 컬럼명, 충돌 in [
    ("대상자명", 대상자명충돌),
    ("대상자구분", 대상자구분충돌)
]:
    
    print(f'{컬럼명} 충돌 ----')
    display(충돌체크용.loc[충돌[충돌 > 1].index])

대상자명 충돌 ----


,,index,채무자키,관할법원,대상자명,대상자구분,당사포함여부,종국결과,종국결과일,최신화일,이름키구분
새채무자키,사건번호,,,,,,,,,,


대상자구분 충돌 ----


,,index,채무자키,관할법원,대상자명,대상자구분,당사포함여부,종국결과,종국결과일,최신화일,이름키구분
새채무자키,사건번호,,,,,,,,,,


#### 3. 정렬 및 중복제거

In [31]:
# 중복제거전 확인.(모두 없어야)
print('전산등록된 채무자 파산건인데 이름키 없는 건')
display(파산[(파산.대상자구분.isin(['채무자|보증인']))&((파산.이름키_=="")|파산.이름키_.isna())])
print('대상자구분, 이름키구분 모두 없는 건')
display(파산[(파산.대상자구분=="")&(파산.이름키구분=="")])

전산등록된 채무자 파산건인데 이름키 없는 건


,채무자키,새채무자키,대상자명,대상자구분,관할법원,사건번호,당사포함여부,종국결과,종국결과일,이름키_,이름키구분,수정전사건번호,접수일,종료일,확정일,최신화일,법조치키,법조치구분


대상자구분, 이름키구분 모두 없는 건


,채무자키,새채무자키,대상자명,대상자구분,관할법원,사건번호,당사포함여부,종국결과,종국결과일,이름키_,이름키구분,수정전사건번호,접수일,종료일,확정일,최신화일,법조치키,법조치구분


#### 4. 최종점검(동일인 다중사건)

In [32]:
# 정렬 및 중복제거
print('중복제거전', len(파산))

    # 정렬 : 아래서 이름키로 작업할 게 남아서 이름키로 정렬
파산 = 파산.sort_values(["이름키_"])

    # 중복제거
파산 = 파산.drop_duplicates(subset=["새채무자키","사건번호"])
print('병합 및 중복제거후',len(파산))

중복제거전 1511
병합 및 중복제거후 873


In [33]:
# 동일인 다중 사건처리(이름키_ 중복) : by 사건연도, 수정전사건번호, 법조치구분
    # 필요 객체 생성
조회용파산 = 파산.set_index(["이름키_"]) # group객체로는 여러 인덱스를 한번에 조회할 수 없음
파산grouped = 파산.groupby(["이름키_"])

    # 필요 칼럼 추가
파산["사건연도"] = 파산['사건번호'].str[:4]
파산["사건번호_고유수"] = (파산grouped["사건번호"].transform(lambda x: x.nunique(dropna=True)))
파산["사건연도_고유수"] = (파산grouped["사건연도"].transform(lambda x: x.nunique(dropna=True)))
파산['상속재산파산여부'] = (파산["법조치구분"].eq("상속재산파산").groupby(파산["이름키_"]).transform("any"))



    # 1) 대상자구분이 상속재산파산이고, 사건번호의 연도가 같으면 하면 사건을 제거 : 상속재산파산건이 파산으로 법조치에 추가등록되어있거나, 채무자에 등록되어있으면 여기서 걸러짐
삭제conds = ((파산['상속재산파산여부']) & (파산["사건번호_고유수"] > 1) & (파산['사건연도_고유수']==1) & (파산["수정전사건번호"].str.contains('하단'))) 
상속재산파산_하면ind = 파산[삭제conds].set_index(["이름키_"]).index

파산 = 파산[~삭제conds]
상속재산파산_하면삭제후건수 = len(파산)


    # 2) 사건번호의 연도가 같으면 하단사건을 제거
삭제conds = (~(파산['상속재산파산여부']) & (파산["사건번호_고유수"] > 1) & (파산['사건연도_고유수']==1) & (파산["사건번호"].str.contains('하단')))  
일반파산_하단ind = 파산[삭제conds].set_index(["이름키_"]).index

파산 = 파산[~삭제conds].drop(columns=["사건연도", "사건번호_고유수", "사건연도_고유수", "상속재산파산여부"]).reset_index(drop=True)
일반파산_하단삭제후건수 = len(파산)

    # 3) 삭제건 확인
print(f'삭제전건수:{len(조회용파산)}, 상속재산파산_하면삭제후건수:{상속재산파산_하면삭제후건수}, 일반파산_하단삭제후건수:{일반파산_하단삭제후건수}')
print("")
print('상속재산파산_하면 삭제건 확인')
display(조회용파산[조회용파산.index.isin(상속재산파산_하면ind)][["채무자키","대상자구분","대상자명","법조치구분","사건번호","수정전사건번호","종국결과","종국결과일","당사포함여부","법조치키"]])
print('일반파산_하단 삭제건 확인-다 하면으로 바꿨으니 없어야 맞음')
display(조회용파산[조회용파산.index.isin(일반파산_하단ind)][["채무자키","대상자구분","대상자명","법조치구분","사건번호","수정전사건번호","종국결과","종국결과일","당사포함여부","법조치키"]])
    
# 미사용코드 -------------------------------------------------------------------------
# 괜히 다중사건 아닌 것처럼 보여서 오히려 혼동 & 이런 경우 거의 없고, 코드 여러모로 복잡해져서 아래 코드는 사용하지 않기로
# 잔존 다중사건 중 최근 파산사건만 조회하기(이전에 취하,기각,불허가 였거나, 당사누락이었다면)
# 조회제외conds = ((파산["사건번호_고유수"] > 1) & (파산['사건연도_고유수'] > 1) & (파산['종국결과'].str.contains('취하|기각|불허가|불가', na=False) | 파산["당사포함여부"].str.contains('누락', na=False)))
# 파산[파산.duplicated(subset='이름키_', keep=False) & ~조회제외conds]
# ----------------------------------------------------------------------------------

삭제전건수:873, 상속재산파산_하면삭제후건수:869, 일반파산_하단삭제후건수:869

상속재산파산_하면 삭제건 확인


,채무자키,대상자구분,대상자명,법조치구분,사건번호,수정전사건번호,종국결과,종국결과일,당사포함여부,법조치키
이름키_,,,,,,,,,,
n1025683공정식,20409906,채무자,공정식,상속재산파산,2023하단10132,2023하면10132,<NA>,<NA>,포함,20908588
n1025683공정식,20409906,채무자,공정식,NaN,2023하면10132,2023하단10132,NaN,NaN,<NA>,NaN
n1027209신경섭,20410672,채무자,신경섭,NaN,2024하면20418,2024하단20418,<NA>,<NA>,포함,NaN
n1027209신경섭,20410672,채무자,신경섭,상속재산파산,2024하단20418,2024하면20418,<NA>,<NA>,포함,20908295
n1038068황세동,20461536,채무자,황세동,상속재산파산,2023하단21,NaN,<NA>,<NA>,포함,20910449
n1038068황세동,20461536,채무자,황세동,NaN,2023하면21,2023하단21,NaN,NaN,<NA>,NaN
n1050462조혜영,20430257,채무자,조혜영,NaN,2022하면1344,2022하단1344,NaN,NaN,<NA>,NaN
n1050462조혜영,20430257,채무자,조혜영,상속재산파산,2022하단1344,NaN,<NA>,<NA>,포함,20910800


일반파산_하단 삭제건 확인-다 하면으로 바꿨으니 없어야 맞음


,채무자키,대상자구분,대상자명,법조치구분,사건번호,수정전사건번호,종국결과,종국결과일,당사포함여부,법조치키
이름키_,,,,,,,,,,


In [34]:
# 삭제 후 잔존 다중사건 확인
print('(전산, df 수정필요)오타인 경우가 대부분')
파산[파산.duplicated(subset='이름키_', keep=False)].reset_index().set_index("이름키_")[["채무자키","법조치키","관할법원","사건번호","index","종국결과"]]

(전산, df 수정필요)오타인 경우가 대부분


,채무자키,법조치키,관할법원,사건번호,index,종국결과
이름키_,,,,,,
n1003367황옥희,20415865,NaN,창원지방법원,2008하면765,93,인용
n1003367황옥희,20415865,NaN,창원지방법원,2008하면764,94,NaN
n1006125이종환,20414292,20681598,서울회생법원,2009하면30075,148,신청취하
n1006125이종환,20414292,20912076,서울회생법원,2024하면572,149,<NA>
n1024043노경환,20425491,NaN,의정부지방법원,2024하면20217,265,NaN
n1024043노경환,20425491,20913481,의정부지방법원,2024하면20127,266,인용
n1025802이병주,20410685,20915600,대전지방법원,2024하면10259,284,신청취하
n1025802이병주,20410685,NaN,대전지방법원,2024하면10715,285,NaN
n1039382도예진,20465329,NaN,대구지방법원,2024하면11013,488,NaN


In [35]:
# 삭제 및 수정
파산 = 파산.drop(index=[30])

#### 5. 최종정렬 및 엑셀출력

In [36]:
# 정렬순서 지정 : 채무자, 보증인, 상속인, 삭제보증인
정렬순서 = ["채무자", "보증인", "상속인", "삭제보증인"]
파산["대상자구분"] = pd.Categorical(
    파산["대상자구분"],
    categories=정렬순서,
    ordered=True
)

파산 = 파산.sort_values(["대상자구분", "새채무자키", "채무자키"])

# 출력
print(f'파산기준데이터 {len(파산)}건 엑셀출력')
functions.save_df_to_excel_underline(파산, join(wd, basedate, "파산_기준데이터_"+basedate+".xlsx"),10)
파산.to_pickle(join(wd, basedate, "pkl", "파산_기준데이터_"+basedate+".pkl"))

파산기준데이터 868건 엑셀출력
